In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import  get_peft_model
from langchain.prompts import PromptTemplate
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from IPython.display import Markdown, display

ModuleNotFoundError: No module named 'langchain'

In [35]:
### 微调后的模型名

new_model = "/root/autodl-tmp/llama-2-7b-correct—question"
model_name = "/root/autodl-tmp/Llama-2-7b-chat-hf" 

In [40]:
def format_text(example):
    """ fill inputs in promt for a sample  """
    example = example['text']
    example = eval(example)
    options = {}
    option = []
    for j in range(len(example['options'])):
        option = []
        i = example['options'][j]
        for t in range(len(i)):
            s = chr(ord('A') + t) + '.' + i[t]
            option.append(s)
        options['question'+str(j+1)] = option
    questions = {}
    answers = []
    for i in range(len(example['answers'])):
        s = ord(example['answers'][i]) - ord('A')
        answers.append(options['question'+str(i+1)][s])
    for j in range(len(example['questions'])):
        questions[j+1] = 'question:'+example['questions'][j]+'\n'+'options:'+ str(options['question'+str(j+1)])
    text = prompt.format(instruction='Answer the questions according to the article and the options.',
                         input='Article:'+example['article']+'\n\n' +'Questions:' + str(questions) + '\n',
                         output=str(answers))
    return {'text':text}


In [14]:
dataset = load_dataset(path="text",data_dir='/root/autodl-tmp/race', split=['train', 'test'])

Resolving data files:   0%|          | 0/19366 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1389 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1407 [00:00<?, ?it/s]

In [42]:
dataset[1][1]

{'text': '{"answers": ["A", "C", "D", "B", "C", "D"], "options": [["his daughter was just born", "he was thirteen years old", "his mother died of cancer", "his wife was out of work"], ["was once saved by his lovely guide dog", "is taken good care of by the government", "thinks it\'s natural to have setbacks in life", "believes it takes a little longer to do things"], ["the effective traffic system", "the kindness of New Yorkers", "the loyalty of his guide dog", "his will to overcome difficulty"], ["Sympathy counts most.", "Ability comes first.", "Preference for the blind.", "Easy job for the weak."], ["make plenty of money", "win his supervisor\'s trust", "live like a normal person", "complete his daily task"], ["deep thinking", "weak motivation", "special talents", "great independence"]], "questions": ["Mr. Torres became blind when  _  .", "Mr. Torres does not feel sorry for himselfbecause he   _  .", "The description of Mr. Torres\'s long trip to work shows   _  .", "What is the prin

In [27]:
train_dataset = dataset[1].map(format_text)
train_dataset[1]

Map:   0%|          | 0/1407 [00:00<?, ? examples/s]

{'text': '\nInstruction: Answer the questions according to the article and the options.\n\nInput: Article:You could feel sorry for Alberto Torres, who is blind. The last thing he remembers seeing was his daughter being born 13 years ago. Then the world went blank; he can only imagine what his only child looks like now, as a teenaged honor student. Bad luck is no stranger to this warm and thoughtful 37-year-old man. His mother died of cancer when he was 4, and his father, who was often ill, had to give him up to the care of the state when he was 11. He later worked for 19 years in a workshop making household goods, deathly boring work. Earlier this month, Alberto Torres\'s wife, who had just been laid off from her job, had to have a breast removed due to cancer and now faces a year of radiation treatments. Things seemed always to go from almost incredibly bad to worse. Even Mr. Torres\'s good luck has a dark side: Five years ago, his lovely guide dog pulled him out of the path of a truc

In [17]:
# prepare template 
template = """
Instruction: {instruction}\n
Input: {input}\n
Output: {output}<\s>"""


prompt = PromptTemplate(template=template, input_variables=['instruction', 'input','output'])
prompt

PromptTemplate(input_variables=['instruction', 'input', 'output'], output_parser=None, partial_variables={}, template='\nInstruction: {instruction}\n\nInput: {input}\n\nOutput: {output}<\\s>', template_format='f-string', validate_template=True)

In [28]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [29]:
lora_config = LoraConfig.from_pretrained('/root/autodl-tmp/llama-2-7b-correct/')

In [30]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="results_Correct_questions",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [31]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [32]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

/root/miniconda3/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1407 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.271700
50,2.299500
75,2.320500
100,2.323600
125,2.324100
150,2.247200
175,2.319000
200,2.277900
225,2.326000
250,2.279700


TrainOutput(global_step=352, training_loss=2.3013067678971724, metrics={'train_runtime': 2105.2811, 'train_samples_per_second': 1.337, 'train_steps_per_second': 0.167, 'total_flos': 4.139712041877504e+16, 'train_loss': 2.3013067678971724, 'epoch': 2.0})

In [36]:
trainer.model.save_pretrained(new_model)
new_model

'/root/autodl-tmp/llama-2-7b-correct—question'

In [37]:
lora_config = LoraConfig.from_pretrained('/root/autodl-tmp/llama-2-7b-correct—question')
new_model = get_peft_model(model, lora_config)
new_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (

In [38]:
prompt = """Instruction: According to the articles and options provide answer this question.\n\nInput: Article:You could feel sorry for Alberto Torres, who is blind. The last thing he remembers seeing was his daughter being born 13 years ago. Then the world went blank; he can only imagine what his only child looks like now, as a teenaged honor student. Bad luck is no stranger to this warm and thoughtful 37-year-old man. His mother died of cancer when he was 4, and his father, who was often ill, had to give him up to the care of the state when he was 11. He later worked for 19 years in a workshop making household goods, deathly boring work. Earlier this month, Alberto Torres\'s wife, who had just been laid off from her job, had to have a breast removed due to cancer and now faces a year of radiation treatments. Things seemed always to go from almost incredibly bad to worse. Even Mr. Torres\'s good luck has a dark side: Five years ago, his lovely guide dog pulled him out of the path of a truck. Mr. Torres was not hurt. The dog was killed.\nBut Mr. Torres does not feel sorry for himself. "These are just little bumps you have to go over in your life," he said. At 5 a.m. on a recent morning, we caught up with Mr. Torres at a subway stop in Brooklyn, New York, near where he lives in a third-floor apartment (with no elevator). He had been up since 3 a.m., feeding his new dog, making coffee, getting ready. "When you\'re blind, it takes a little longer to do things," he said.\nMr. Torres was beginning his two-hour trip to his job developing film in the X-ray department of the emergency room of the Bronx hospital. He would take the G train to Queens Plaza station to the R train, heading towards Manhattan. He would then ride the R train to 59th Street where he would walk upstairs to switch to the Number 6 train. At one point along the journey, he might chat with a stranger. At another, someone would pat his dog, calling him by name. People offered assistance, even seats. At 125th Street, Mr. Torres would transfer to the Number 4 train by crossing the platform. At 149th Street, he would go down to the Number 2. He would take that to East 180th Street where he nearly always has a long wait for his final train, to Pelham Parkway. Then he and his dog would walk 20 minutes to the hospital.\nIt was a hard job to come by. Before he got the job, Mr. Torres was determined to escape the workshop run by an organization dedicated to help people who can\'t see. He wanted a job developing X-ray film, something that everyone must do in the dark. He had to handle the long trip, as well as the work. "Our philosophy here is that blind people can do just about anything except drive buses," it was the thinking about disabled people at the Bronx hospital. "We find what a person can do rather than what he can\'t do," said the hospital\'s director.\nOne day a while ago marked the first anniversary of Mr. Torres\'s hiring. He developed 150 or so X-rays, his usual output, to celebrate. Mr. Torres works by himself in a small, dark room that smells of chemicals. He cannot wear gloves, because he needs to feel. It is hard work, related to emergency of lives. His immediate supervisor says he trusts him 100%. Mr. Torres makes $20,000 a year. But his motivation goes beyond money. "If I start feeling like a victim, that makes me bitter. And why be bitter? That makes you go into a hole and stay there." he said. "I\'m not doing anything out of the ordinary," insisted Mr. Torres as he quickly completed the task.\n\nQuestions:{\'question1\': \'Mr. Torres became blind when  _  .\', \'question2\': \'Mr. Torres does not feel sorry for himselfbecause he   _  .\', \'question3\': "The description of Mr. Torres\'s long trip to work shows   _  .", \'question4\': \'What is the principle of the hospital in employing a worker?\', \'question5\': \'Mr. Torres works very hard in order to   _  .\', \'question6\': \'In the eyes of the writer, Alberto Torres is a man of   _  .\'}\nOptions: {\'question1\': [\'(A)his daughter was just born\', \'(B)he was thirteen years old\', \'(C)his mother died of cancer\', \'(D)his wife was out of work\'], \'question2\': [\'(A)was once saved by his lovely guide dog\', \'(B)is taken good care of by the government\', "(C)thinks it\'s natural to have setbacks in life", \'(D)believes it takes a little longer to do things\'], \'question3\': [\'(A)the effective traffic system\', \'(B)the kindness of New Yorkers\', \'(C)the loyalty of his guide dog\', \'(D)his will to overcome difficulty\'], \'question4\': [\'(A)Sympathy counts most.\', \'(B)Ability comes first.\', \'(C)Preference for the blind.\', \'(D)Easy job for the weak.\'], \'question5\': [\'(A)make plenty of money\', "(B)win his supervisor\'s trust", \'(C)live like a normal person\', \'(D)complete his daily task\'], \'question6\': [\'(A)deep thinking\', \'(B)weak motivation\', \'(C)special talents\', \'(D)great independence\']}"""
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=1500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
answer_new = (result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [44]:
prompt = """'text': '\nInstruction: Answer the questions according to the article and the options.\n\nInput: Article:You could feel sorry for Alberto Torres, who is blind. The last thing he remembers seeing was his daughter being born 13 years ago. Then the world went blank; he can only imagine what his only child looks like now, as a teenaged honor student. Bad luck is no stranger to this warm and thoughtful 37-year-old man. His mother died of cancer when he was 4, and his father, who was often ill, had to give him up to the care of the state when he was 11. He later worked for 19 years in a workshop making household goods, deathly boring work. Earlier this month, Alberto Torres\'s wife, who had just been laid off from her job, had to have a breast removed due to cancer and now faces a year of radiation treatments. Things seemed always to go from almost incredibly bad to worse. Even Mr. Torres\'s good luck has a dark side: Five years ago, his lovely guide dog pulled him out of the path of a truck. Mr. Torres was not hurt. The dog was killed.\nBut Mr. Torres does not feel sorry for himself. "These are just little bumps you have to go over in your life," he said. At 5 a.m. on a recent morning, we caught up with Mr. Torres at a subway stop in Brooklyn, New York, near where he lives in a third-floor apartment (with no elevator). He had been up since 3 a.m., feeding his new dog, making coffee, getting ready. "When you\'re blind, it takes a little longer to do things," he said.\nMr. Torres was beginning his two-hour trip to his job developing film in the X-ray department of the emergency room of the Bronx hospital. He would take the G train to Queens Plaza station to the R train, heading towards Manhattan. He would then ride the R train to 59th Street where he would walk upstairs to switch to the Number 6 train. At one point along the journey, he might chat with a stranger. At another, someone would pat his dog, calling him by name. People offered assistance, even seats. At 125th Street, Mr. Torres would transfer to the Number 4 train by crossing the platform. At 149th Street, he would go down to the Number 2. He would take that to East 180th Street where he nearly always has a long wait for his final train, to Pelham Parkway. Then he and his dog would walk 20 minutes to the hospital.\nIt was a hard job to come by. Before he got the job, Mr. Torres was determined to escape the workshop run by an organization dedicated to help people who can\'t see. He wanted a job developing X-ray film, something that everyone must do in the dark. He had to handle the long trip, as well as the work. "Our philosophy here is that blind people can do just about anything except drive buses," it was the thinking about disabled people at the Bronx hospital. "We find what a person can do rather than what he can\'t do," said the hospital\'s director.\nOne day a while ago marked the first anniversary of Mr. Torres\'s hiring. He developed 150 or so X-rays, his usual output, to celebrate. Mr. Torres works by himself in a small, dark room that smells of chemicals. He cannot wear gloves, because he needs to feel. It is hard work, related to emergency of lives. His immediate supervisor says he trusts him 100%. Mr. Torres makes $20,000 a year. But his motivation goes beyond money. "If I start feeling like a victim, that makes me bitter. And why be bitter? That makes you go into a hole and stay there." he said. "I\'m not doing anything out of the ordinary," insisted Mr. Torres as he quickly completed the task.\n\nQuestions:{1: "question:Mr. Torres became blind when  _  .\\noptions:[\'A.his daughter was just born\', \'B.he was thirteen years old\', \'C.his mother died of cancer\', \'D.his wife was out of work\']", 2: \'question:Mr. Torres does not feel sorry for himselfbecause he   _  .\\noptions:[\\\'A.was once saved by his lovely guide dog\\\', \\\'B.is taken good care of by the government\\\', "C.thinks it\\\'s natural to have setbacks in life", \\\'D.believes it takes a little longer to do things\\\']\', 3: "question:The description of Mr. Torres\'s long trip to work shows   _  .\\noptions:[\'A.the effective traffic system\', \'B.the kindness of New Yorkers\', \'C.the loyalty of his guide dog\', \'D.his will to overcome difficulty\']", 4: "question:What is the principle of the hospital in employing a worker?\\noptions:[\'A.Sympathy counts most.\', \'B.Ability comes first.\', \'C.Preference for the blind.\', \'D.Easy job for the weak.\']", 5: \'question:Mr. Torres works very hard in order to   _  .\\noptions:[\\\'A.make plenty of money\\\', "B.win his supervisor\\\'s trust", \\\'C.live like a normal person\\\', \\\'D.complete his daily task\\\']\', 6: "question:In the eyes of the writer, Alberto Torres is a man of   _  .\\noptions:[\'A.deep thinking\', \'B.weak motivation\', \'C.special talents\', \'D.great independence\']"}\n\n\nOutput: [\'A.his daughter was just born\', "C.thinks it\'s natural to have setbacks in life", \'D.his will to overcome difficulty\', \'B.Ability comes first.\', \'C.live like a normal person\', \'D.great independence\']<\\s>'"""
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

'<s>[INST] \'text\': \'\nInstruction: Answer the questions according to the article and the options.\n\nInput: Article:You could feel sorry for Alberto Torres, who is blind. The last thing he remembers seeing was his daughter being born 13 years ago. Then the world went blank; he can only imagine what his only child looks like now, as a teenaged honor student. Bad luck is no stranger to this warm and thoughtful 37-year-old man. His mother died of cancer when he was 4, and his father, who was often ill, had to give him up to the care of the state when he was 11. He later worked for 19 years in a workshop making household goods, deathly boring work. Earlier this month, Alberto Torres\'s wife, who had just been laid off from her job, had to have a breast removed due to cancer and now faces a year of radiation treatments. Things seemed always to go from almost incredibly bad to worse. Even Mr. Torres\'s good luck has a dark side: Five years ago, his lovely guide dog pulled him out of the p

In [27]:
print('origin sentence:', prompt)
print('model:',answer_old)
print('new_model:', answer_new)

origin sentence: Instruction: According to the articles and options provide answer this question.

Input: Article:You could feel sorry for Alberto Torres, who is blind. The last thing he remembers seeing was his daughter being born 13 years ago. Then the world went blank; he can only imagine what his only child looks like now, as a teenaged honor student. Bad luck is no stranger to this warm and thoughtful 37-year-old man. His mother died of cancer when he was 4, and his father, who was often ill, had to give him up to the care of the state when he was 11. He later worked for 19 years in a workshop making household goods, deathly boring work. Earlier this month, Alberto Torres's wife, who had just been laid off from her job, had to have a breast removed due to cancer and now faces a year of radiation treatments. Things seemed always to go from almost incredibly bad to worse. Even Mr. Torres's good luck has a dark side: Five years ago, his lovely guide dog pulled him out of the path of 